In [2]:
import requests
import pandas as pd
import numpy as np
import json
import matplotlib
from pandas import json_normalize



In [6]:
# Create function to transform cleaned DataFrame from movies under threshold vote amount into new DF "df_movies"

def movie_vote_amount(vote_count):
    df_movies = df_clean.loc[df_clean["movie_vote_count"] >= vote_count]
    return df_movies

movie_vote_amount(10)

,director_name,director_gender,director_id,director_popularity,movie_release_date,movie_vote_average,movie_vote_count,movie_original_language,movie_title,movie_id
0,Dodo Abashidze,2,560142,0.728,1985-12-01,7.1,14.0,ka,The Legend of Suram Fortress,103590.0
1,Dodo Abashidze,2,560142,0.728,1968-12-22,5.8,12.0,ka,Don't Grieve!,27922.0
2,Dodo Abashidze,2,560142,0.728,1969-12-28,6.1,11.0,ka,Pirosmani,88302.0
3,George Abbott,2,2004,0.6,1957-08-29,6.3,21.0,en,The Pajama Game,40867.0
4,George Abbott,2,2004,0.6,1958-07-23,6.9,17.0,en,Damn Yankees!,41253.0
...,...,...,...,...,...,...,...,...,...,...
9237,Terry Zwigoff,0,17447,1.094,1994-09-10,7.8,132.0,en,Crumb,26564.0
9238,Andrzej Żuławski,2,32082,0.6,1981-05-27,7.4,382.0,fr,Possession,21484.0
9239,Andrzej Żuławski,2,32082,0.6,1988-10-09,8.3,46.0,pl,On the Silver Globe,27072.0
9240,Andrzej Żuławski,2,32082,0.6,1975-02-12,7.5,43.0,fr,That Most Important Thing: Love,3476.0


In [87]:
# Joining AwardDF to Df_Clean
directors_all = pd.read_pickle("directors_all.pkl")
directors_all



,Year,Winner,Film,Nominee,Type
0,1928,True,Two Arabian Knights,Lewis Milestone,Academy Award
1,1928,False,Speedy,Ted Wilde,Academy Award
2,1928,True,7th Heaven,Frank Borzage,Academy Award
3,1928,False,Sorrell and Son,Herbert Brenon,Academy Award
4,1928,False,The Crowd,King Vidor,Academy Award
...,...,...,...,...,...
807,2019,True,1917,Sam Mendes,Golden Globe
808,2019,False,Parasite,Bong Joon Ho,Golden Globe
809,2019,False,Joker,Todd Phillips,Golden Globe
810,2019,False,"Irishman, The",Martin Scorsese,Golden Globe


In [88]:
# Changing nominee name to "director_name"
directors_all.rename(columns={'Nominee':'director_name'}, inplace=True)
directors_all

,Year,Winner,Film,director_name,Type
0,1928,True,Two Arabian Knights,Lewis Milestone,Academy Award
1,1928,False,Speedy,Ted Wilde,Academy Award
2,1928,True,7th Heaven,Frank Borzage,Academy Award
3,1928,False,Sorrell and Son,Herbert Brenon,Academy Award
4,1928,False,The Crowd,King Vidor,Academy Award
...,...,...,...,...,...
807,2019,True,1917,Sam Mendes,Golden Globe
808,2019,False,Parasite,Bong Joon Ho,Golden Globe
809,2019,False,Joker,Todd Phillips,Golden Globe
810,2019,False,"Irishman, The",Martin Scorsese,Golden Globe


In [89]:
# Cleaning directors_all DF, as the excel file provided some formatting issues
# 1. Check the amount of affected rows
(directors_all["director_name"].str.replace(" ", "").str.replace("\t","").str.isalpha().value_counts())


True     776
False     36
Name: director_name, dtype: int64

In [95]:
# After doing the merge, many names were found to not match with the TMDB DataFrame

directors_all_gender = (directors_all.merge(right=df_clean, 
                                   how="left", 
                                   on="director_name").groupby("director_name").agg({"director_gender":"max"}))
directors_all_gender.loc[directors_all_gender["director_gender"].isna()==True]

,director_gender
director_name,
\tDamien Chazelle,NaN
Adam McKay,NaN
Adam McKay\t,NaN
Barry Jenkins,NaN
Barry Jenkins\t,NaN
...,...
W. S. Van Dyke,NaN
W. S. Van Dyke,NaN
"Warren Beatty, Buck Henry",NaN


In [104]:
# So Fuzzy Logic was required - Matching based on similarity.

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    df_1 is the left table to join
    df_2 is the right table to join
    key1 is the key column of the left table
    key2 is the key column of the right table
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    limit is the amount of matches that will get returned, these are sorted high to low
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1


directors_all_matched = fuzzy_merge(directors_all, df_clean, "director_name", "director_name", threshold=80)

,Year,Winner,Film,director_name,Type,matches
0,1928,True,Two Arabian Knights,Lewis Milestone,Academy Award,"Lewis Milestone, Lewis Milestone"
1,1928,False,Speedy,Ted Wilde,Academy Award,
2,1928,True,7th Heaven,Frank Borzage,Academy Award,"Frank Borzage, Frank Borzage"
3,1928,False,Sorrell and Son,Herbert Brenon,Academy Award,"Herbert Brenon, Herbert Brenon"
4,1928,False,The Crowd,King Vidor,Academy Award,"King Vidor, King Vidor"
...,...,...,...,...,...,...
807,2019,True,1917,Sam Mendes,Golden Globe,"Sam Mendes, Sam Mendes"
808,2019,False,Parasite,Bong Joon Ho,Golden Globe,"Bong Joon-ho, Bong Joon-ho"
809,2019,False,Joker,Todd Phillips,Golden Globe,
810,2019,False,"Irishman, The",Martin Scorsese,Golden Globe,"Martin Scorsese, Martin Scorsese"


In [105]:
directors_all_matched = fuzzy_merge(directors_all, df_clean, "director_name", "director_name", threshold=80)

In [127]:
director_names = (directors_all_matched["matches"].str.split(", ",n=1,expand=True).reset_index())
director_names

,index,0,1
0,0,Lewis Milestone,Lewis Milestone
1,1,,None
2,2,Frank Borzage,Frank Borzage
3,3,Herbert Brenon,Herbert Brenon
4,4,King Vidor,King Vidor
...,...,...,...
807,807,Sam Mendes,Sam Mendes
808,808,Bong Joon-ho,Bong Joon-ho
809,809,,None
810,810,Martin Scorsese,Martin Scorsese


In [ ]:
# Renaming columns and adding columns to acquire redquired DataFrame

In [124]:
directors_all_matched = directors_all_matched.assign(TMDBnames=director_names[1])

In [126]:
directors_all_matched = directors_all_matched.rename(columns={"director_name":"name","TMDBnames":"director_name"})
directors_all_matched


,Year,Winner,Film,name,Type,matches,name
0,1928,True,Two Arabian Knights,Lewis Milestone,Academy Award,"Lewis Milestone, Lewis Milestone",Lewis Milestone
1,1928,False,Speedy,Ted Wilde,Academy Award,,None
2,1928,True,7th Heaven,Frank Borzage,Academy Award,"Frank Borzage, Frank Borzage",Frank Borzage
3,1928,False,Sorrell and Son,Herbert Brenon,Academy Award,"Herbert Brenon, Herbert Brenon",Herbert Brenon
4,1928,False,The Crowd,King Vidor,Academy Award,"King Vidor, King Vidor",King Vidor
...,...,...,...,...,...,...,...
807,2019,True,1917,Sam Mendes,Golden Globe,"Sam Mendes, Sam Mendes",Sam Mendes
808,2019,False,Parasite,Bong Joon Ho,Golden Globe,"Bong Joon-ho, Bong Joon-ho",Bong Joon-ho
809,2019,False,Joker,Todd Phillips,Golden Globe,,None
810,2019,False,"Irishman, The",Martin Scorsese,Golden Globe,"Martin Scorsese, Martin Scorsese",Martin Scorsese


In [130]:
# Had issues with renaming, as initial DF was botched. Way to manually rename the column by index, didn't work.
directors_all_matched = directors_all_matched.rename(columns={directors_all_matched.columns[6]:"director_name"})
directors_all_matched

,Year,Winner,Film,director_name,Type,matches,director_name
0,1928,True,Two Arabian Knights,Lewis Milestone,Academy Award,"Lewis Milestone, Lewis Milestone",Lewis Milestone
1,1928,False,Speedy,Ted Wilde,Academy Award,,None
2,1928,True,7th Heaven,Frank Borzage,Academy Award,"Frank Borzage, Frank Borzage",Frank Borzage
3,1928,False,Sorrell and Son,Herbert Brenon,Academy Award,"Herbert Brenon, Herbert Brenon",Herbert Brenon
4,1928,False,The Crowd,King Vidor,Academy Award,"King Vidor, King Vidor",King Vidor
...,...,...,...,...,...,...,...
807,2019,True,1917,Sam Mendes,Golden Globe,"Sam Mendes, Sam Mendes",Sam Mendes
808,2019,False,Parasite,Bong Joon Ho,Golden Globe,"Bong Joon-ho, Bong Joon-ho",Bong Joon-ho
809,2019,False,Joker,Todd Phillips,Golden Globe,,None
810,2019,False,"Irishman, The",Martin Scorsese,Golden Globe,"Martin Scorsese, Martin Scorsese",Martin Scorsese


In [133]:
# Manually renamed all columns in order to fix.
directors_all_matched.columns = ["Year","Winner","Film","Name","Type","matches","director_name"]
directors_all_matched

,Year,Winner,Film,Name,Type,matches,director_name
0,1928,True,Two Arabian Knights,Lewis Milestone,Academy Award,"Lewis Milestone, Lewis Milestone",Lewis Milestone
1,1928,False,Speedy,Ted Wilde,Academy Award,,None
2,1928,True,7th Heaven,Frank Borzage,Academy Award,"Frank Borzage, Frank Borzage",Frank Borzage
3,1928,False,Sorrell and Son,Herbert Brenon,Academy Award,"Herbert Brenon, Herbert Brenon",Herbert Brenon
4,1928,False,The Crowd,King Vidor,Academy Award,"King Vidor, King Vidor",King Vidor
...,...,...,...,...,...,...,...
807,2019,True,1917,Sam Mendes,Golden Globe,"Sam Mendes, Sam Mendes",Sam Mendes
808,2019,False,Parasite,Bong Joon Ho,Golden Globe,"Bong Joon-ho, Bong Joon-ho",Bong Joon-ho
809,2019,False,Joker,Todd Phillips,Golden Globe,,None
810,2019,False,"Irishman, The",Martin Scorsese,Golden Globe,"Martin Scorsese, Martin Scorsese",Martin Scorsese


In [152]:
# Finally did the joining to match genders from TMDB DF to the directors all DF
directors_all_gender = (directors_all_matched.merge(right=df_clean, 
                                   how="left", 
                                   on="director_name").groupby(["director_name","Year"]).agg({"director_gender":"max","Winner":np.count_nonzero}))
directors_all_gender

director_gender  Winner
director_name    Year                         
Adrian Lyne      1987                2       0
Akira Kurosawa   1985                2       0
Alan J. Pakula   1976                2       0
Alan Parker      1978                2       0
                 1988                2       0
...                                ...     ...
Woody Allen      1989                2       0
                 1994                2       0
                 2005                2       0
                 2011                2       0
Édouard Molinaro 1979                2       0

[546 rows x 2 columns]

In [153]:
directors_all_gender.to_pickle("directors_genders_awards.pkl")

In [ ]:
directors_genders_awards = pd.read_pickle("directors_genders_awards.pkl")
directors_genders_awards

In [154]:
directors_all_matched.to_pickle("directors_all_matched.pkl")